In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
from tqdm import tqdm   #for文の進捗を確認

In [2]:
# ペルソナを意識して情報収集の条件を変更
# 目黒区、2K, 2DK, 2LDK

load_url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13110&cb=0.0&ct=9999999&et=9999999&md=05&md=06&md=07&ts=1&ts=2&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2="

res = requests.get(load_url)
res.encoding = "utf-8"
soup = BeautifulSoup(res.text, "html.parser")

In [3]:
# ページ数の取得
pages = soup.find("ol", class_="pagination-parts")
num_of_pages = int(pages.find_all("li")[-1].text)

In [4]:
# データフレーム作成
rental_property_datas = pd.DataFrame(columns=["名前", "住所", "最寄り駅１","最寄り駅２","最寄り駅３", "築年数", "建物全体の階数", "階数", "賃料", "管理費", "敷金", "礼金", "間取り", "面積", "URL" ])

In [5]:
for p in tqdm(range(num_of_pages)):
# for p in tqdm(range(2)):

    # topのページは"&page=1"を付けても開ける
    page_url = load_url + "&page=" + str(p+1)
    res = requests.get(page_url)
    res.encoding = "utf-8"
    soup = BeautifulSoup(res.text, "html.parser")
    
    # サーバー負荷を避けるため1s遅延
    time.sleep(1)

    # 「建物ごとに表示」ページから物件毎の情報を取得
    cassetitems = soup.find_all("div", class_="cassetteitem")

    for i in range(len(cassetitems)):
        
        # 上の部分（建物情報）
        details = cassetitems[i].find_all("div", class_="cassetteitem-detail")
        
        for ii in range(len(details)):
            name = details[ii].find("div", class_="cassetteitem_content-title").text
            address = details[ii].find("li", class_="cassetteitem_detail-col1").text
            _stations = details[ii].find_all("div", class_="cassetteitem_detail-text")
            station1 = _stations[0].text
            station2 = _stations[1].text
            station3 = _stations[2].text
            _col3 = details[ii].find_all("li", class_="cassetteitem_detail-col3")
            building_age = _col3[0].find_all("div")[0].text
            number_of_floors = _col3[0].find_all("div")[1].text
            
            # 下の部分（部屋の情報）
            items = cassetitems[i].find("div", class_="cassetteitem-item")
            tbodys = items.find_all("tbody")
            for iii in range(len(tbodys)):
                _tds = tbodys[iii].find_all("td")
                floor = _tds[2].text.replace('\r','').replace('\n','').replace('\t','')
                rent = _tds[3].find("span", class_="cassetteitem_other-emphasis ui-text--bold").text
                maintenance_fee = _tds[3].find("span", class_="cassetteitem_price cassetteitem_price--administration").text
                deposit = _tds[4].find("span", class_="cassetteitem_price cassetteitem_price--deposit").text
                gratuity = _tds[4].find("span", class_="cassetteitem_price cassetteitem_price--gratuity").text
                layout = _tds[5].find("span", class_="cassetteitem_madori").text
                area = _tds[5].find("span", class_="cassetteitem_menseki").text
                url = "https://suumo.jp" + _tds[8].find("a").get("href")
                
                # DataFrameにまとめた後、rental_property_datasへ追加する
                _d = pd.DataFrame()
                _d["名前"] = [name]
                _d["住所"] = [address]
                _d["最寄り駅１"] = [station1]
                _d["最寄り駅２"] = [station2]
                _d["最寄り駅３"] = [station3]
                _d["築年数"] = [building_age]
                _d["建物全体の階数"] = [number_of_floors]
                _d["階数"] = [floor]
                _d["賃料"] = [rent]
                _d["管理費"] = [maintenance_fee]
                _d["敷金"] = [deposit]
                _d["礼金"] = [gratuity]
                _d["間取り"] = [layout]
                _d["面積"] = [area]
                _d["URL"] = [url]

                rental_property_datas = pd.concat([rental_property_datas, _d], ignore_index=True)

                

  0%|          | 0/23 [00:00<?, ?it/s]

100%|██████████| 23/23 [00:55<00:00,  2.41s/it]


In [6]:
df = rental_property_datas.copy()

# csvへ保存
df.to_csv('SUUMO_meguro_2K2DK2LDK_raw.csv', index=False, encoding='utf-8-sig')

# 重複判定(データをクレンジングせず重複判定してみる)

df_raw = df[df[["住所","賃料","管理費","間取り","建物全体の階数","階数","面積","敷金","礼金"]].duplicated()]
len(df_raw)

317

In [7]:
# クレンジングに使用した処理を関数化

def replace_B_to_minus(floor: str) -> int:
    """地下を示すBを"-"に置き換える（"階数"で使用）

    Args:
        floor (str): X階数の"X"だけを渡す_

    Returns:
        int: 特に、BX -> -X で返す
    """
    if "B" in floor :
        floor = int(floor.replace("B","-"))
    else :
        floor = int(floor)

    return floor

def replace_man_yen_to_int(money: str) -> int:
    """X.X万円をintに変換する

    Args:
        money (str): X.X万円

    Returns:
        int: XX000 (円)
    """
    if "-" in money :
        money = int(0)
    
    elif "." in money :
        money = int(money.split(".")[0]) * 10000 +  int(float("0."+money.split(".")[1].replace("万円", ""))*10000)
    else :
        money = int(money.replace("万円", ""))* 10000

    return money

def replace_yen_to_int(money: str) -> int:
    """X円をintに変換する

    Args:
        money (str): X円

    Returns:
        int: X (円)
    """
    if "-" in money :
        money = int(0)
    else :
        money = int(money.replace("円", ""))

    return money

In [8]:
# if文で作成してたものをラムダ式に整理

rental_property_datas["築年数"] = rental_property_datas["築年数"].apply(lambda x : int(0) if x=="新築" else int(x.replace("築","").replace("年","")))

# "建物全体の階数を"最小と最大のコラムに直す
rental_property_datas["建物階数_最小"] = rental_property_datas["建物全体の階数"].apply(lambda x : int(x.split("地上")[0].replace("地下","-")) if "地下" in x else int(0))
rental_property_datas["建物階数_最大"] = rental_property_datas["建物全体の階数"].apply(lambda x : int(x.split("地上")[1].replace("階建","")) if "地下" in x else x.replace("階建",""))

# "階数"を"最小と最大のコラムに直す
rental_property_datas["階数_最小"] = rental_property_datas["階数"].apply(lambda x : replace_B_to_minus(x.split("-")[0]) if "-" in x else replace_B_to_minus(x.replace("階", "")))
rental_property_datas["階数_最大"] = rental_property_datas["階数"].apply(lambda x : replace_B_to_minus(x.split("-")[1].replace("階", "")) if "-" in x else replace_B_to_minus(x.replace("階", "")))

rental_property_datas["賃料"] = rental_property_datas["賃料"].apply(lambda x: replace_man_yen_to_int(x))
rental_property_datas["敷金"] = rental_property_datas["敷金"].apply(lambda x : replace_man_yen_to_int(x))
rental_property_datas["礼金"] = rental_property_datas["礼金"].apply(lambda x : replace_man_yen_to_int(x))

rental_property_datas["管理費"] = rental_property_datas["管理費"].apply(lambda x : replace_yen_to_int(x))

rental_property_datas["面積"] = rental_property_datas["面積"].apply(lambda x : x.replace("m2",""))
# rental_property_datas["間取り"] = rental_property_datas["間取り"]  # そのままでよい

In [9]:
# 重複判定

df = rental_property_datas
df_cleansing= df[df[["住所","賃料","管理費","間取り","建物階数_最小","建物階数_最大","階数_最小","階数_最大","面積","敷金","礼金"]].duplicated()]
print(len(df_cleansing))

df_cleansing.to_csv('SUUMO_meguro_2K2DK2LDK.csv', index=False, encoding='utf-8-sig')

df_cleansing

317


,名前,住所,最寄り駅１,最寄り駅２,最寄り駅３,築年数,建物全体の階数,階数,賃料,管理費,敷金,礼金,間取り,面積,URL,建物階数_最小,建物階数_最大,階数_最小,階数_最大
66,パーク・ハイム目黒碑文谷,東京都目黒区碑文谷１,東急東横線/学芸大学駅 歩15分,東急東横線/都立大学駅 歩18分,東急目黒線/西小山駅 歩18分,26,地下1地上8階建,3階,218000,0,218000,218000,2LDK,66.54,https://suumo.jp/chintai/jnc_000086968363/?bc=...,-1,8,3,3
69,東急東横線 学芸大学駅 4階建 築33年,東京都目黒区目黒本町２,東急東横線/学芸大学駅 歩10分,東急目黒線/西小山駅 歩19分,東急目黒線/武蔵小山駅 歩18分,33,4階建,4階,113000,3000,113000,113000,2DK,36.73,https://suumo.jp/chintai/jnc_000080222531/?bc=...,0,4,4,4
70,東急東横線 学芸大学駅 4階建 築33年,東京都目黒区目黒本町２,東急東横線/学芸大学駅 歩10分,東急目黒線/西小山駅 歩19分,東急目黒線/武蔵小山駅 歩18分,33,4階建,4階,113000,3000,113000,113000,2K,36.73,https://suumo.jp/chintai/jnc_000084537264/?bc=...,0,4,4,4
72,東急目黒線 不動前駅 3階建 築29年,東京都目黒区下目黒４,東急目黒線/不動前駅 歩13分,東急目黒線/武蔵小山駅 歩18分,ＪＲ山手線/目黒駅 歩19分,29,3階建,3階,115000,5000,115000,115000,2K,35.72,https://suumo.jp/chintai/jnc_000084909190/?bc=...,0,3,3,3
82,パークサイドハウス,東京都目黒区目黒本町２,東急東横線/学芸大学駅 歩14分,東急目黒線/武蔵小山駅 歩14分,東急目黒線/西小山駅 歩17分,47,2階建,1階,79000,2000,79000,79000,2K,30,https://suumo.jp/chintai/jnc_000085494167/?bc=...,0,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097,スウィフト東山,東京都目黒区東山３,東急田園都市線/池尻大橋駅 歩1分,京王井の頭線/駒場東大前駅 歩18分,京王井の頭線/神泉駅 歩17分,31,地下1地上5階建,4-5階,285000,15000,285000,285000,2LDK,76.23,https://suumo.jp/chintai/jnc_000085951025/?bc=...,-1,5,4,5
1104,目黒ハイツ,東京都目黒区青葉台３,東急東横線/中目黒駅 歩10分,東京メトロ日比谷線/中目黒駅 歩10分,東急田園都市線/池尻大橋駅 歩7分,57,10階建,5階,160000,9000,320000,80000,2LDK,51.97,https://suumo.jp/chintai/jnc_000086526578/?bc=...,0,10,5,5
1106,東急東横線 学芸大学駅 4階建 築33年,東京都目黒区柿の木坂３,東急東横線/学芸大学駅 歩15分,東急東横線/都立大学駅 歩18分,東急田園都市線/三軒茶屋駅 歩26分,33,4階建,1階,230000,0,230000,0,2LDK,60.7,https://suumo.jp/chintai/jnc_000086347898/?bc=...,0,4,1,1
1107,目黒台スカイマンション,東京都目黒区三田２,ＪＲ山手線/恵比寿駅 歩11分,ＪＲ山手線/目黒駅 歩10分,東急東横線/代官山駅 歩19分,45,11階建,2階,250000,0,250000,250000,2LDK,61.9,https://suumo.jp/chintai/jnc_000085773596/?bc=...,0,11,2,2
